Aby uruchomić poniższe notatniki, jeśli jeszcze tego nie zrobiłeś, musisz ustawić klucz openai w pliku .env jako `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Następnie załadujemy Indeks Osadzeń do DataFrame Pandas. Indeks Osadzeń jest przechowywany w pliku JSON o nazwie `embedding_index_3m.json`. Indeks Osadzeń zawiera osadzenia dla każdej z transkrypcji YouTube do końca października 2023 roku.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Następnie utworzymy funkcję o nazwie `get_videos`, która będzie wyszukiwać w Indeksie Osadzeń (Embedding Index) na podstawie zapytania. Funkcja zwróci 5 najlepszych filmów najbardziej podobnych do zapytania. Funkcja działa w następujący sposób:

1. Najpierw tworzona jest kopia Indeksu Osadzeń.
2. Następnie obliczane jest Osadzenie (Embedding) dla zapytania za pomocą OpenAI Embedding API.
3. Potem w Indeksie Osadzeń tworzona jest nowa kolumna o nazwie `similarity`. Kolumna `similarity` zawiera kosinusową miarę podobieństwa między Osadzeniem zapytania a Osadzeniem dla każdego segmentu wideo.
4. Następnie Indeks Osadzeń jest filtrowany według kolumny `similarity`. Indeks Osadzeń jest filtrowany tak, aby zawierał tylko filmy, które mają kosinusową miarę podobieństwa większą lub równą 0,75.
5. Na koniec Indeks Osadzeń jest sortowany według kolumny `similarity` i zwracane jest 5 najlepszych filmów.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ta funkcja jest bardzo prosta, po prostu wyświetla wyniki zapytania wyszukiwania.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Najpierw indeks osadzeń jest ładowany do DataFrame Pandas.
2. Następnie użytkownik jest proszony o wpisanie zapytania.
3. Następnie wywoływana jest funkcja `get_videos`, aby przeszukać indeks osadzeń pod kątem zapytania.
4. Na koniec wywoływana jest funkcja `display_results`, aby wyświetlić wyniki użytkownikowi.
5. Następnie użytkownik jest proszony o wpisanie kolejnego zapytania. Proces ten trwa, aż użytkownik wpisze `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.pl.png)

Zostaniesz poproszony o wpisanie zapytania. Wpisz zapytanie i naciśnij enter. Aplikacja zwróci listę filmów, które są istotne dla zapytania. Aplikacja zwróci również link do miejsca w filmie, gdzie znajduje się odpowiedź na pytanie.

Oto kilka zapytań do wypróbowania:

- Czym jest Azure Machine Learning?
- Jak działają splotowe sieci neuronowe?
- Czym jest sieć neuronowa?
- Czy mogę używać Jupyter Notebooks z Azure Machine Learning?
- Czym jest ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zastrzeżenie**:  
Niniejszy dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mimo że dokładamy starań, aby tłumaczenie było jak najbardziej precyzyjne, prosimy mieć na uwadze, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w języku źródłowym należy traktować jako źródło wiarygodne i autorytatywne. W przypadku informacji o kluczowym znaczeniu zalecane jest skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
